In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_tweets_anuFYb8.csv', 'train_E6oV3lV.csv']


In [2]:
import numpy as np
import pandas as pd

In [34]:
df = pd.read_csv('../input/train_E6oV3lV.csv')
df['label'] = df['label'].map({0: 2, 1: 1})
print(df.head())
# print(df['label'].head())
df = df.drop('id', axis=1)

# print(df[:100])
# zero = 0
# one = 0
# for i in df['label']:
#     if(i==0):
#         zero+=1
#     else:
#         one+=1
    
# print(zero,one)

   id                        ...                                                                      tweet
0   1                        ...                           @user when a father is dysfunctional and is s...
1   2                        ...                          @user @user thanks for #lyft credit i can't us...
2   3                        ...                                                        bihday your majesty
3   4                        ...                          #model   i love u take with u all the time in ...
4   5                        ...                                     factsguide: society now    #motivation

[5 rows x 3 columns]


In [35]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import copy

In [36]:

train, test = train_test_split(df, test_size=0.2)
print(len(train))
print(len(test))
print(train.head())

print(type(train))

25569
6393
       label                                              tweet
20529      2   @user âbe with those who help your being.â...
17833      2     @user a   comic to make you #smile and laugh. 
27050      2  lmao @ guys liking random girls pictures like ...
17683      2  #goodmorning âï¸ð¥ð #neymar #playing ð...
25301      2  happy at work c.onference: right mindset leads...
<class 'pandas.core.frame.DataFrame'>


In [37]:
train_df = copy.deepcopy(train)
test_df = copy.deepcopy(test)

# concatenate column 1 and column 2 as one text
# print(train_df[1])

# convert string to lower case
train_texts = train_df['tweet'].values
train_texts = [s.lower() for s in train_texts]

test_texts = test_df['tweet'].values
test_texts = [s.lower() for s in test_texts]

# print(train_texts.head())
# print(train_texts)


In [38]:

# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

# -----------------------Skip part start--------------------------
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789 ,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
# -----------------------Skip part end----------------------------

# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

# =======================Get classes================
train_classes = train_df['label'].values
train_class_list = [x - 1 for x in train_classes]

test_classes = test_df['label'].values
test_class_list = [x - 1 for x in test_classes]

from keras.utils import to_categorical

train_classes = to_categorical(train_class_list)
test_classes = to_categorical(test_class_list)

In [39]:
print(len(test_classes))
print(tk.word_index)

6393
{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '0': 27, '1': 28, '2': 29, '3': 30, '4': 31, '5': 32, '6': 33, '7': 34, '8': 35, '9': 36, ' ': 37, ',': 38, ';': 39, '.': 40, '!': 41, '?': 42, ':': 43, "'": 44, '"': 45, '/': 46, '\\': 47, '|': 48, '_': 49, '@': 50, '#': 51, '$': 52, '%': 53, '^': 54, '&': 55, '*': 56, '~': 57, '`': 58, '+': 59, '-': 60, '=': 61, '<': 62, '>': 63, '(': 64, ')': 65, '[': 66, ']': 67, '{': 68, '}': 69, 'UNK': 70}


In [40]:


vocab_size = len(tk.word_index)
vocab_size



70

In [41]:
embedding_weights = [] #(71, 70)
embedding_weights.append(np.zeros(vocab_size)) # first row is pad

for char, i in tk.word_index.items(): # from index 1 to 70
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)


In [42]:
print(embedding_weights.shape) # first row all 0 for PAD, 69 char, last row for UNK
embedding_weights

(71, 70)


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [43]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

In [44]:
# parameter 
input_size = 1014
# vocab_size = 69
embedding_size = 70
conv_layers = [[256, 7, 3], 
               [256, 7, 3], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, -1], 
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = 2
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'

In [45]:
# Embedding layer Initialization
embedding_layer = Embedding(vocab_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

In [46]:
# Model 

# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding 
x = embedding_layer(inputs)
# Conv 
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x) 
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x) # Final shape=(None, 34, 256)
x = Flatten()(x) # (None, 8704)
# Fully connected layers 
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x) # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # Adam, categorical_crossentropy
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 1014)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1014, 70)          4970      
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 1008, 256)         125696    
_________________________________________________________________
activation_13 (Activation)   (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 336, 256)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 330, 256)          459008    
_________________________________________________________________
activation_14 (Activation)   (None, 330, 256)          0         
__________

In [49]:
# 1000 training samples and 100 testing samples
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes

In [50]:


# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=10,
          verbose=2)



Train on 25569 samples, validate on 6393 samples
Epoch 1/10
 - 13s - loss: 0.1876 - acc: 0.9315 - val_loss: 0.1360 - val_acc: 0.9496
Epoch 2/10
 - 13s - loss: 0.1238 - acc: 0.9555 - val_loss: 0.1172 - val_acc: 0.9571
Epoch 3/10
 - 13s - loss: 0.0919 - acc: 0.9678 - val_loss: 0.1198 - val_acc: 0.9546
Epoch 4/10
 - 13s - loss: 0.0641 - acc: 0.9774 - val_loss: 0.1294 - val_acc: 0.9537
Epoch 5/10
 - 13s - loss: 0.0593 - acc: 0.9795 - val_loss: 0.1109 - val_acc: 0.9629
Epoch 6/10
 - 13s - loss: 0.0362 - acc: 0.9870 - val_loss: 0.1243 - val_acc: 0.9611
Epoch 7/10
 - 13s - loss: 0.0374 - acc: 0.9867 - val_loss: 0.1432 - val_acc: 0.9604
Epoch 8/10
 - 13s - loss: 0.0213 - acc: 0.9923 - val_loss: 0.1498 - val_acc: 0.9620
Epoch 9/10
 - 13s - loss: 0.0153 - acc: 0.9947 - val_loss: 0.1718 - val_acc: 0.9617
Epoch 10/10
 - 13s - loss: 0.0126 - acc: 0.9960 - val_loss: 0.3250 - val_acc: 0.9565
